In [1]:
import boto3
import pandas as pd
import io
import re
import os
from collections import defaultdict
from tqdm import tqdm

# AWS S3 Configuration
bucket_name = 'pepper-dataset'
source_prefix = 'county-env-data/county_env_NetCDF4/'
target_prefix = 'Temp/'

s3 = boto3.client('s3')

# Step 1: List all CSV files under the source prefix
print("Listing all CSV files in S3...")
paginator = s3.get_paginator('list_objects_v2')
page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=source_prefix)
files = []
for page in page_iterator:
    files += [content['Key'] for content in page.get('Contents', []) if content['Key'].endswith('.csv')]

Listing all CSV files in S3...


In [14]:
# Step 2: Organize files by state and year
organized = defaultdict(lambda: defaultdict(list))

for file_key in files:
    # Example key: county-env-data/county_env_NetCDF4/Arkansas_Jackson/Arkansas_Jackson_2014-08_part1.csv
    try:
        filename = os.path.basename(file_key)  # e.g., Arkansas_Jackson_2014-08_part1.csv
        parts = filename.split('_')
        if len(parts) < 3:
            continue
        state_county = f"{parts[0]}_{parts[1]}"  # Arkansas_Jackson
        year_month = parts[2]  # 2014-08
        year = year_month.split('-')[0]
        state = parts[0]  # Arkansas

        organized[state][year].append(file_key)

    except Exception as e:
        print(f"[WARN] Skipping unrecognized key: {file_key}, error: {e}")
        continue

In [ ]:
# Step 3: Process and upload
for state, year_map in tqdm(organized.items(), desc="Processing states"):
    for year, file_list in tqdm(year_map.items(), desc=f"Processing {state} {year}", leave=False):
        monthly_data = defaultdict(dict)

        for file_key in file_list:
            try:
                filename = os.path.basename(file_key)
                parts = filename.split('_')
                state_county = f"{parts[0]}_{parts[1]}"
                ym = parts[2]
                part = parts[3].split('.')[0]  # e.g., part1

                obj = s3.get_object(Bucket=bucket_name, Key=file_key)
                df = pd.read_csv(io.BytesIO(obj['Body'].read()))
                df['datetime'] = pd.to_datetime(df['datetime'])
                monthly_data[(state_county, ym)][part] = df
            except Exception as e:
                print(f"[ERROR] Failed to read {file_key}: {e}")
                continue

        merged_all = []
        for (state_county, ym), parts in monthly_data.items():
            try:
                if 'part1' in parts and 'part2' in parts:
                    merged = pd.merge(parts['part1'], parts['part2'], on='datetime', suffixes=('_part1', '_part2'))
                elif 'part1' in parts:
                    merged = parts['part1']
                elif 'part2' in parts:
                    merged = parts['part2']
                else:
                    continue
                merged['county'] = state_county
                merged_all.append(merged)
            except Exception as e:
                print(f"[ERROR] Failed to merge {state_county} {ym}: {e}")
                continue

        if merged_all:
            final_df = pd.concat(merged_all, ignore_index=True)
            filename = f"{state}_counties_{year}.csv"
            local_path = f"/tmp/{filename}"

            try:
                final_df.to_csv(local_path, index=False)

                # Ensure file is not empty
                if not os.path.exists(local_path) or os.path.getsize(local_path) == 0:
                    raise Exception(f"[EMPTY] Local file is missing or empty: {local_path}")

                # Upload to S3
                s3.upload_file(local_path, bucket_name, f"{target_prefix}{filename}")
                print(f"[UPLOAD ✅] s3://{bucket_name}/{target_prefix}{filename} ({len(final_df)} rows)")

                # Double check upload
                resp = s3.list_objects_v2(Bucket=bucket_name, Prefix=f"{target_prefix}{filename}")
                found = any(obj['Key'] == f"{target_prefix}{filename}" for obj in resp.get('Contents', []))
                if not found:
                    raise Exception(f"[S3-MISS] File not found in S3 after upload: {filename}")

                # Remove local temp file
                os.remove(local_path)

            except Exception as e:
                print(f"[ERROR ❌] Upload failed for {filename}: {e}")

Processing Arkansas 2023:   4%|▍         | 1/24 [00:15<05:57, 15.53s/it]

[UPLOAD ✅] s3://pepper-dataset/Temp/Arkansas_counties_2000.csv (35136 rows)



Processing Arkansas 2023:   8%|▊         | 2/24 [00:31<05:44, 15.68s/it]

[UPLOAD ✅] s3://pepper-dataset/Temp/Arkansas_counties_2001.csv (35040 rows)



Processing Arkansas 2023:  12%|█▎        | 3/24 [00:47<05:34, 15.94s/it]

...

[UPLOAD ✅] s3://pepper-dataset/Temp/Illinois_counties_2003.csv (385440 rows)



Processing Illinois 2023:  21%|██        | 5/24 [12:04<45:20, 143.17s/it]

[UPLOAD ✅] s3://pepper-dataset/Temp/Illinois_counties_2004.csv (386496 rows)



Processing Illinois 2023:  25%|██▌       | 6/24 [14:24<42:37, 142.11s/it]

[UPLOAD ✅] s3://pepper-dataset/Temp/Illinois_counties_2005.csv (385440 rows)


In [12]:
merged_all

[]